In [2]:
# Create a new workspace
from azureml.core import Workspace, Experiment, Run

#subscription_id='{azure-subscription-id}'
ws = Workspace.create(
            name = 'AMLWorkspace',
            subscription_id='485f9fa6-0221-4dbf-8074-a202426a29bf', 
            resource_group='rgAMLSLearnworkspace',
            create_resource_group=True,
            location='eastus2'
)
print("Workspace is built")

WorkspaceException: WorkspaceException:
	Message: Workspace with name 'AMLWorkspace' already exists under resource group with name 'rgAMLSLearnworkspace'.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Workspace with name 'AMLWorkspace' already exists under resource group with name 'rgAMLSLearnworkspace'."
    }
}

In [3]:
# Or use an existed workspace
ws = Workspace.get(name="AMLWorkspace",
               subscription_id='485f9fa6-0221-4dbf-8074-a202426a29bf',
               resource_group='rgAMLSLearnworkspace')

In [4]:
# Step 1: Build a remote compute target
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# Step 1-1: name the cluster and set the minimal and maximal number of nodes 
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpucluster")
min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 3)

# Step 1-2: choose environment variables 
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")

provisioning_config = AmlCompute.provisioning_configuration(
    vm_size = vm_size, min_nodes = min_nodes, max_nodes = max_nodes)

# create the cluster
compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

print('Compute target created')

Compute target created


In [ ]:
# Load MNIST data
import os
import urllib.request

#create a folder for the dataset
os.makedirs('./data', exist_ok = True)

# load dataset to the directory--as you can see, you must load train sets and test sets separately
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', filename='./data/train-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', filename='./data/train-labels.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', filename='./data/test-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', filename='./data/test-labels.gz')

print('Done')

In [5]:
#Step 2: Upload data to workspace by using get_default_datastore()
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='mnist', overwrite=True, show_progress=True)

print('Done')

Uploading an estimated of 4 files
Uploading ./data\test-images.gz
Uploading ./data\test-labels.gz
Uploading ./data\train-images.gz
Uploading ./data\train-labels.gz
Uploaded ./data\test-labels.gz, 1 files out of an estimated total of 4
Uploaded ./data\train-labels.gz, 2 files out of an estimated total of 4
Uploaded ./data\test-images.gz, 3 files out of an estimated total of 4
Uploaded ./data\train-images.gz, 4 files out of an estimated total of 4
Uploaded 4 files
Done


In [6]:
# Step 3: Create a folder to save the code
import os

# create the folder
folder_training_script = './trial_model_mnist'
os.makedirs(folder_training_script, exist_ok=True)

print('Done')

Done
